**3. Density Operator master equations**

**3.1. Introduction to Lindblad master equation**

The Gorini-Kossakowski-Sudarshan-Lindblad (GKSL) master equation, often known as the *Lindblad* master equation:

$$\dot{\rho}(t) = -\frac{i}{\hbar} [H, \rho(t)] + \sum_k \gamma_k \left(L_k\rho(t)L_k^\dagger - \frac{1}{2}\Bigl\{L_k^\dagger L_k, \rho(t)\Bigr\}\right)$$

where $\{L_k\}$ are the *Lindblad* operators representing some non-unitary processes like relaxation or decoherence that occur at some rates $\{\gamma_k\}$.

Like the Hamiltonian generates coherent dynamics, the Lindblad operators generate incoherent transitions in the space of states. Unlike the Hamiltonian, they do not need to be hermitian. For example, a decay transition from some excited state $|e\rangle$ to some ground state $|g\rangle$ is mediated by the Lindblad operator

$$L_\downarrow = |g\rangle\langle e|$$

Indeed, when we apply $L_\downarrow$ to $|e\rangle$ we obtain $|g\rangle = L_\downarrow|e\rangle$. Note that $L_\downarrow^\dagger = |e\rangle\langle g| \neq L_\downarrow$.

This Lindblad equation is used to approxiamte the evolution of the density operator of a system $\text{S}$ with Hamiltonian $H$ that is weakly coupled to a Markovian (memory-less) environment, and is a general form for a **completely positive and trace-preserving (CPTP) Markovian** and **time-homogeneous map** for the evolution of the system's density operator $\rho$.

**3.2 The Liouville superoperator**